In [6]:
from dotenv import load_dotenv
from typing import Dict, List, Tuple
import guidance
import os

load_dotenv()

api_key = os.getenv("api_key")
organization = os.getenv("organization")

gpt4 = guidance.llms.OpenAI(api_key=api_key, organization=organization, model="gpt-4")
davinci = guidance.llms.OpenAI(api_key=api_key, organization=organization, model="text-davinci-003")

In [7]:
domain = "SACHS Protein-Signaling Network"

variables = [
    "Plcg", 
    "PIP3", 
    "PIP2", 
    "P38", 
    "Jnk", 
    "PKA", 
    "PKC", 
    "Raf", 
    "Mek", 
    "Erk", 
    "Akt"
]

edges = [
    ("Plcg", "PIP3"),
    ("Plcg", "PIP2"),
    ("PIP3", "PIP2"),
    ("PKC", "Raf"),
    ("PKC", "PKA"),
    ("PKC", "Jnk"),
    ("PKC", "P38"),
    ("PKC", "Mek"),
    ("PKA", "Raf"),
    ("PKA", "Mek"),
    ("PKA", "Erk"),
    ("PKA", "Akt"),
    ("PKA", "Jnk"),
    ("PKA", "P38"),
    ("Raf", "Mek"),
    ("Mek", "Erk"),
    ("Erk", "Akt"),
]

In [8]:
import sys
sys.path.append("..")
print(sys.path)

from suggesters import ModelSuggester
model = ModelSuggester()

['/home/antonkulaga/.local/share/JetBrains/Toolbox/apps/IDEA-U/ch-0/232.9559.62.plugins/python/helpers-pro/jupyter_debug', '/home/antonkulaga/.local/share/JetBrains/Toolbox/apps/IDEA-U/ch-0/232.9559.62.plugins/python/helpers/pydev', '/home/antonkulaga/sources/pywhy-llm/notebooks/sachs_proteins', '/home/antonkulaga/sources/pywhy-llm', '/home/antonkulaga/micromamba/envs/pywhy-llm/lib/python310.zip', '/home/antonkulaga/micromamba/envs/pywhy-llm/lib/python3.10', '/home/antonkulaga/micromamba/envs/pywhy-llm/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/sources/indexpaper', '/home/antonkulaga/micromamba/envs/pywhy-llm/lib/python3.10/site-packages', '..', '..']


In [9]:
llm_descriptions = {}
for var in variables:
    llm_descriptions[var] = 0


llm_edges = {}
for var_a in variables:
    for var_b in variables:
        if var_a != var_b:
            llm_edges[(var_a, var_b)] = 0


In [10]:
for var in variables:
    llm_descriptions = model.suggest_description(llm=gpt4, variable=var)

TypeError: ModelSuggester.suggest_description() got an unexpected keyword argument 'variable'

In [13]:
# i = 1
# for var_a in variables:
#     for var_b in variables:
#         if var_a != var_b:
                
#             if llm_edges[(var_a, var_b)] == 0 and llm_edges[(var_b, var_a)] == 0:
#                 id = "pair" + str(i)
#                 i += 1

#                 if llm_descriptions[var_a] == 0 and llm_descriptions[var_b] == 0:
#                     direction = model.suggest_relationship(llm=gpt4, variable_a=var_a, variable_b=var_b, description_a=llm_descriptions[var_a], description_b=llm_descriptions[var_b])
#                 else:
#                     direction = model.suggest_relationship(llm=gpt4, variable_a=var_a, variable_b=var_b, domain=domain)
                    
#                 if direction=="R" and (var_a, var_b) in edges:
#                     correct_direction = 1
#                 elif direction=="L" and (var_b, var_a) in edges:
#                     correct_direction = 1
#                 elif direction=="N" and (var_a, var_b) not in edges and (var_b, var_a) not in edges:
#                     correct_direction = 1
#                 else:
#                     correct_direction= 0


#                 llm_edges[(var_a, var_b)] = { "pair_id": id, 
#                                     "var_a": var_a,
#                                     "var_b": var_b, 
#                                     "correct_direction": correct_direction
#                                     }

In [14]:
# import csv
# import copy

# # CSV file name
# csv_file = "gpt-4_results_straight_prompt_w_descriptions.csv"

# # Define the CSV file's header (column names)
# header = ["CorrectACauseB", "CorrectBCauseA", "PairID", "VarA", "VarB", "GroundTruth"]

# # Write the data to the CSV file
# with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
#     writer = csv.DictWriter(file, fieldnames=header)
#     writer.writeheader() 
#     for pair_id, values in llm_edges.items():
#         writer.writerow(values)

# print(f"CSV file '{csv_file}' has been created.")


# Debugging

In [6]:
def build_two_sided_relationship_program(
        use_domain="",
        use_description=False,
        ask_reference=False,
        consider_feedback_loop=False,
    ):
    assistant = """{{#assistant~}} {{gen 'output' temperature=temperature}} {{~/assistant}}"""

    # set systen
    system = ""
    if use_domain == "":
        system = """{{#system~}} You are a helpful assistant on causal reasoning. Your goal is to answer questions about cause and effect in a factual and concise way. {{~/system}}"""
    else:
        system = """{{#system~}} You are a helpful assistant on causal reasoning and """ + use_domain + """. Your goal is to answer questions about cause and effect in a factual and concise way. {{~/system}}"""

    # set user
    user = """"""
    if use_description:
        user += """{{#user~}} Which of the following cause and effect situations between {{variable_a}}, where {{description_a}}, and {{variable_b}}, where {{description_b}}, make most factual sense?"""

    # no variable description
    else:
        user += """{{#user~}} Which of the following cause and effect situations between {{variable_a}} and {{variable_b}} make most factual sense?"""

    # consider "both" in answer option
    if consider_feedback_loop:
        user += """ 
        A. Changing {{variable_a}} changes {{variable_b}}?
        B. Changing {{variable_b}} changes {{variable_a}}?
        C. Both A and B.
        D. None.
    """

    # don't consider "both" in answer options
    else:
        user += """ 
        A. Changing {{variable_a}} changes {{variable_b}}?
        B. Changing {{variable_b}} changes {{variable_a}}?
        C. None.
    """

    # request research paper support
    if ask_reference:
        user += """What are three research papers that discuss each of these variables? What do they say about the relationship they may or may not have? You are to provide the paper title and a one sentence summary each paper's argument. Then use those arguments as reference to identify which of the cause and effect situations between {{variable_a}} and {{variable_b}} make most factual sense."""

        # fourth option in examples
        if consider_feedback_loop:
            user += """Answer A, B, C, or D.
        Within one sentence, you are to think step-by-step to make sure that you have the right answer. Provide your final answer within the tags, <answer>A/B/C/D</answer>, and your references within the tags <reference>Author, Title, Year of publication</reference. 
        \n\n\n----------------\n\n\nExample of output structure: <answer>A</answer>\n<reference>Author, Title, Year of publication</reference>\n\n\n----------------\n\n\nExample of output structure: <answer>B</answer>\n<reference>Author, Title, Year of publication</reference>----------------\n\n\nExample of output structure: <answer>C</answer>\n<reference>Author, Title, Year of publication</reference>----------------\n\n\nExample of output structure: <answer>D</answer>\n<reference>Author, Title, Year of publication</reference> {{~/user}}"""

        # three options in examples
        else:
            user += """Answer A, B, or C.
        Within one sentence, you are to think step-by-step to make sure that you have the right answer. Provide your final answer within the tags, <answer>A/B/C/D</answer>, and your references within the tags <reference>Author, Title, Year of publication</reference. 
        \n\n\n----------------\n\n\nExample of output structure: <answer>A</answer>\n<reference>Author, Title, Year of publication</reference>\n\n\n----------------\n\n\nExample of output structure: <answer>B</answer>\n<reference>Author, Title, Year of publication</reference>----------------\n\n\nExample of output structure: <answer>C</answer>\n<reference>Author, Title, Year of publication</reference> {{~/user}}"""

    # Don't request research paper support
    else:
        user += """Think carefully and factually to identify which of the cause and effect situations between {{variable_a}} and {{variable_b}} makes most factual sense."""

        # consider four examples
        if consider_feedback_loop:
            user += """Answer A, B, C, D. Within one sentence, you are to think step-by-step to make sure that you have the right answer. Then provide your final answer within the tags, <answer>A/B/C/D</answer>.
            \n\n\n----------------\n\n\nExample of output structure: <answer>A</answer>\n\n\n----------------\n\n\nExample of output structure: <answer>B</answer>----------------\n\n\nExample of output structure: <answer>C</answer>----------------\n\n\nExample of output structure: <answer>D</answer>{{~/user}}"""

        # consider three examples
        else:
            user += """
            Think step-by-step to make sure that you have the right answer. Then provide your final answer within the tags, <answer>A/B/C</answer>.
            \n\n\n----------------\n\n\nExample of output structure: <answer>A</answer>\n\n\n----------------\n\n\nExample of output structure: <answer>B</answer>\n\n\n----------------\n\n\nExample of output structure: <answer>C</answer>{{~/user}}"""

    print(system)
    print(user)
    print(assistant)

    return(system + user + assistant)

In [16]:
program = guidance(build_two_sided_relationship_program(use_domain=domain))

{{#system~}} You are a helpful assistant on causal reasoning and SACHS Protein-Signaling Network. Your goal is to answer questions about cause and effect in a factual and concise way. {{~/system}}
{{#user~}} Which of the following cause and effect situations between {{variable_a}} and {{variable_b}} make most factual sense? 
        A. Changing {{variable_a}} changes {{variable_b}}?
        B. Changing {{variable_b}} changes {{variable_a}}?
        C. None.
    Think carefully and factually to identify which of the cause and effect situations between {{variable_a}} and {{variable_b}} makes most factual sense.
            Think step-by-step to make sure that you have the right answer. Then provide your final answer within the tags, <answer>A/B/C</answer>.
            


----------------


Example of output structure: <answer>A</answer>


----------------


Example of output structure: <answer>B</answer>


----------------


Example of output structure: <answer>C</answer>{{~/user}}
{{#as

In [17]:
for var_a in variables:
    for var_b in variables:
        if var_a != var_b:
            output = program(use_domain=domain, variable_a=var_a, variable_b=var_b, llm=gpt4, temperature=0.3)

Stop program system You are a helpful assistant on causal reasoning and SACHS Protein-Signaling Network. Your goal is to answer questions about cause and effect in a factual and concise way. user Which of the following cause and effect situations between Plcg and PIP3 make most factual sense? 
 A. Changing Plcg changes PIP3 ?
 B. Changing PIP3 changes Plcg ?
 C. None.
 Think carefully and factually to identify which of the cause and effect situations between Plcg and PIP3 makes most factual sense.
 Think step-by-step to make sure that you have the right answer. Then provide your final answer within the tags, <answer>A/B/C</answer>.
 


----------------


Example of output structure: <answer>A</answer>


----------------


Example of output structure: <answer>B</answer>


----------------


Example of output structure: <answer>C</answer> assistant

: 

: 

# Testing appended guidance program string

In [7]:
system = """{{#system~}} You are a really good comedian who can look into the depth of a person, undersand them, and find comedy in their suffering. {{~/system}}"""
user = """{{#user~}} I am learnin about fashion and how to dress myself for who I wish to be. """
user += """I've come to realize how much I love fantasy, medieval, and nordic music (celtic songs), art, scenery, and artistic expression."""
user += """I would like to find a way to add these creative things to my wardrobe in a way where I can comfortably use it at the office.{{~/user}}"""
assistant = """{{#assistant~}} {{gen "output"}} {{~/assistant}}"""

In [8]:
frankenstein=guidance(system + user + assistant)
frankenstein(llm=gpt4)

Stop program system You are a really good comedian who can look into the depth of a person, undersand them, and find comedy in their suffering. user I am learnin about fashion and how to dress myself for who I wish to be. I've come to realize how much I love fantasy, medieval, and nordic music (celtic songs), art, scenery, and artistic expression.I would like to find a way to add these creative things to my wardrobe in a way where I can comfortably use it at the office. assistant

KeyboardInterrupt: 

In [9]:
template = """
{{#system~}} 
You are a really good comedian who can look into the depth of a person, undersand them, and find comedy in their suffering. 
{{~/system}}

{{#user~}} 
I am learnin about fashion and how to dress myself for who I wish to be. 
I've come to realize how much I love fantasy, medieval, and nordic music (celtic songs), art, scenery, and artistic expression.
I would like to find a way to add these creative things to my wardrobe in a way where I can comfortably use it at the office.
{{~/user}}

{{#assistant~}} {{gen "output"}} {{~/assistant}}
"""

In [10]:
not_frank=guidance(template)
not_frank(llm=gpt4)

Stop program 
 {{#system~}} 
You are a really good comedian who can look into the depth of a person, undersand them, and find comedy in their suffering. 
 {{~/system}} 

 {{#user~}} 
I am learnin about fashion and how to dress myself for who I wish to be. 
I've come to realize how much I love fantasy, medieval, and nordic music (celtic songs), art, scenery, and artistic expression.
I would like to find a way to add these creative things to my wardrobe in a way where I can comfortably use it at the office.
 {{~/user}} 

 {{#assistant~}} {{gen "output"}} {{~/assistant}}

In [13]:
template2 = """
You are a really good comedian who can look into the depth of a person, undersand them, and find comedy in their suffering. 
I am learnin about fashion and how to dress myself for who I wish to be. 
I've come to realize how much I love fantasy, medieval, and nordic music (celtic songs), art, scenery, and artistic expression.
I would like to find a way to add these creative things to my wardrobe in a way where I can comfortably use it at the office.
{{gen "output"}}
"""

In [14]:
straight=guidance(template2)
output = straight(llm=davinci)

Stop program 
You are a really good comedian who can look into the depth of a person, undersand them, and find comedy in their suffering. 
I am learnin about fashion and how to dress myself for who I wish to be. 
I've come to realize how much I love fantasy, medieval, and nordic music (celtic songs), art, scenery, and artistic expression.
I would like to find a way to add these creative things to my wardrobe in a way where I can comfortably use it at the office.

KeyboardInterrupt: 